# Sprint 機械学習スクラッチ 決定木

## 【考察】

* どう決定木は作られていくか。
* 以下の条件次第で、木の構成は変わる。
    * 学習方法
    * ハイパーパラメータ
    * 訓練データ

* 今回の決定木は量的変数のみに特化する。
    * カテゴリ変数には「0と1」で代用する。
    


In [163]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
class ScratchDecesionTreeClassifierDepth1():
    """
    深さ1の決定木分類器のスクラッチ実装

    Parameters
    ----------
    verbose : bool
      学習過程を出力する場合はTrue
    """
    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.verbose = verbose
    def fit(self, X, y):
        """
        決定木分類器を学習する
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        """
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass
    def predict(self, X):
        """
        決定木分類器を使いラベルを推定する
        """
        pass
        return

## 【問題1】不純度を求める関数（CART式）

### ノードのジニ不純度を計算する関数を作成してください

* ジニ不純度とは、そのノードでのサンプルのクラスの異なりが同程度存在する確率。
    * 確率が高いとノード内のサンプルが全て、異なるクラスに属している。
        * データが半々なのは悪い分類
    * 確率が低いとノード内のサンプルが全て、同じクラスに属している。
    * ベルヌーイ分布における、全てのクラスの分散の和に相当する。
* ノード内の不純度を最大限減らす（ジニ不純度が低い）素性と閾値の組を選ぶために、ジニ不純度を用いる。
* 不純度が最も低ければジニ不純度の値は0、不純度が高くなればなるほどジニ不純度の値が1に漸近する。（[参照先url](https://qiita.com/3000manJPY/items/ef7495960f472ec14377)）
* 最終的に情報利得Δgainで算出する。
    * 利得が高い特徴と閾値ほど、不純度を最大限減らせる。
    


1. ジニ係数を算出する関数を構築する。
2. ジニ係数を用い、情報利得を算出する関数を構築する。


In [3]:
def gini_coef(*args):
    """""
    ジニ係数を算出する。
    
    Parameters
    ----------
    *args : int
        根ノード内の各特徴量毎のサンプル数（値）を渡す。
        
    return
    ----------
    ジニ係数
    """""
    sample_all = sum(args)
    gini_coef_answer = 0
    for i in range(len(args)):
        gini_coef_answer += np.power(args[i]/sample_all, 2)
    return 1 - gini_coef_answer

In [4]:
# クラス1:サンプル数15, クラス2:サンプル数15 → ジニ不純度0.500
print(f'例題1のジニ不純度: {gini_coef(15, 15)}')
print()
# クラス1:サンプル数15, クラス2:サンプル数15, クラス3:サンプル数15 → ジニ不純度0.667
print(f'例題2のジニ不純度: {gini_coef(15, 15, 15) :.2f}')
print()
# クラス1:サンプル数18, クラス2:サンプル数12 → ジニ不純度0.480
print(f'例題3のジニ不純度: {gini_coef(18, 12)}')
print()
# クラス1:サンプル数30, クラス2:サンプル数0 → ジニ不純度0.000
print(f'例題4のジニ不純度: {gini_coef(30, 0)}')

例題1のジニ不純度: 0.5

例題2のジニ不純度: 0.67

例題3のジニ不純度: 0.48

例題4のジニ不純度: 0.0


## 【問題2】情報利得を求める関数

* 問題1で算出した確率はジニ不純度（ジニ係数）$I(t)$をroot_node $I(p)$として用いる。
* 左右各ノードのサンプル数を引数として情報利得を算出する。

In [87]:
def info_gain(left_node_0, left_node_1, right_node_0, right_node_1):
    """""
    情報利得を算出する。
    
    Parameters
    ----------
    left_node_0 : int
        左ノード内の第1特徴量のサンプル数（値）を渡す。
    left_node_1 : int
        左ノード内の第2特徴量のサンプル数（値）を渡す。
    right_node_0 : int
        右ノード内の第1特徴量のサンプル数（値）を渡す。
    right_node_1 : int
        右ノード内の第2特徴量のサンプル数（値）を渡す。
        
    return
    ----------
    gain : float
        情報利得
    """""    
    sample_all_list = [left_node_0, left_node_1, right_node_0, right_node_1] # パラメータのリスト化
    sample_all = sum(sample_all_list) # 全サンプルの総和
    
    left_all = np.add(left_node_0, left_node_1) # 左分岐の総和
    # print(f'left_all: {left_all}')
    right_all = np.add(right_node_0, right_node_1) # 右分岐の総和
    # print(f'right_all: {right_all}')
    root_node = [left_node_0 + right_node_0, left_node_1 + right_node_1] # 根ノードの左右総和数をリスト化
    # print(f'root_node: {root_node}')
    
    gini_coef_answer = 0 
    for i in range(len(root_node)):
        gini_coef_answer += np.power(root_node[i]/sample_all, 2)
    gini_coef = 1 - gini_coef_answer # ジニ係数
    # print(f'gini_coef: {gini_coef}')
    
    left_node_coef = 1 - (np.power(left_node_0/left_all, 2) + np.power(left_node_1/left_all, 2)) # 左ノードのジニ係数
    # print(f'left_node_coef: {left_node_coef}')
    right_node_coef = 1 - (np.power(right_node_0/right_all, 2) + np.power(right_node_1/right_all, 2)) # 右ノードのジニ係数
    # print(f'right_node_coef: {right_node_coef}')
    gain = gini_coef - ((left_all/sample_all) * left_node_coef) - ((right_all/sample_all) * right_node_coef) # 情報利得の算出
    
    # print(f'左辺: {(left_all/sample_all) * left_node_coef}')
    # print(f'右辺: {(right_all/sample_all) * right_node_coef}')
    return gain

In [6]:
# test
# 左ノードクラス1:サンプル数10, 左ノードクラス2:サンプル数30, 右ノードクラス1:サンプル数20, 右ノードクラス2:サンプル数5 → 情報利得0.143
answer = info_gain(10, 30, 20, 5)
print('例題の情報利得：0.143')
print(f'スクラッチ関数の利得：{answer:.3f}')

例題の情報利得：0.143
スクラッチ関数の利得：0.143


## 【問題3】学習

全サンプルの情報利得を算出する。

1. 説明変数Xから、行列内の要素を一つ抽出する。
2. 抽出した要素を閾値として、全ての説明変数を葉ノードに振り分ける。
    * 閾値以下：左
    * 閾値以上：右
3. 左右の葉ノード内も目的変数yの0, 1の個数を返す。
    * 情報利得算出に必要なのはラベル別の個数！
        * 葉ノードのラベル別個数
            * left_node_1
            * left_node_2
            * right_node_1
            * right_node_2
4. 問題2の関数を用いて、情報利得を算出する。
5. 算出した情報利得を新しい行列内に追記する。
6. 最初に戻り、次のインデックスを抽出する。（繰り返す）
7. 全てのインデックスを基に算出した情報利得を代入した行列を完成させる。
8. 最大値の情報利得を抽出する。
    1. 情報利得を代入した行列内の最大値を抽出する。
    2. 抽出した要素とインデックスをインスタンス化させて、次の推定に活用する。

### サンプルデータ

In [23]:
# Sample data
X = np.array([[1, 2],[3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([0, 1, 1, 0, 1])
# y = [0, 1, 1, 0, 1]
X

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10]])

### 3-1. 学習データ内の特徴量別の行番号、個数の確認

In [219]:
# 目的変数yの1次元行列の内、0のインデックス、1のインデックスを返す。
y_count_0 = np.where(y == 0)[0].tolist()
y_count_1 = np.where(y == 1)[0].tolist()
print(f'インデックス0の行番号：{y_count_0} \nインデックス1の行番号：{y_count_1}')
# 返されたインデックスが、説明変数Xの行数になる。

# 0と1の個数も同時に算出する。（np.uniqueのreturn_countsを使う。）
counts_0_1 = np.unique(y, return_counts=True)[1]
print(f'インデックス0の個数：{counts_0_1[0]}\nインデックス1の個数：{counts_0_1[1]}')

インデックス0の行番号：[0, 3] 
インデックス1の行番号：[1, 2, 4]
インデックス0の個数：2
インデックス1の個数：3


### 3-2. 閾値に基づいて、全サンプルを葉ノードへ振り分ける。（例：閾値 = 4）

In [220]:
threshold = 4

# 0列目が閾値以下に該当する行数
terminal_node_left_row_0 = list(zip(*np.where(X[:, 0] < threshold)))
print(f'0列目が閾値以下（左葉ノード）に該当する行数：{terminal_node_left_row_0}')

# 1列目が閾値以下に該当する行数
terminal_node_left_row_1 = list(zip(*np.where(X[:, 1] < threshold)))
print(f'1列目が閾値以下（左葉ノード）に該当する行数：{terminal_node_left_row_1}')

# 0列目が閾値以上に該当する行数
terminal_node_right_row_0 = list(zip(*np.where(X[:, 0] > threshold)))
print(f'0列目が閾値以上（右葉ノード）に該当する行数：{terminal_node_right_row_0}')

# 1列目が閾値以上に該当する行数
terminal_node_right_row_1 = list(zip(*np.where(X[:, 1] > threshold)))
print(f'1列目が閾値以上（右葉ノード）に該当する行数：{terminal_node_right_row_0}')


0列目が閾値以下（左葉ノード）に該当する行数：[(0,), (1,)]
1列目が閾値以下（左葉ノード）に該当する行数：[(0,)]
0列目が閾値以上（右葉ノード）に該当する行数：[(2,), (3,), (4,)]
1列目が閾値以上（右葉ノード）に該当する行数：[(2,), (3,), (4,)]


### 3-3. 左右の葉ノードへ振り分けた後、各特徴量別の行番号を抽出（重複させないためunique関数を用いる）

In [222]:
left_u = np.unique(np.append(terminal_node_left_row_0, terminal_node_left_row_1))
right_u = np.unique(np.append(terminal_node_right_row_0, terminal_node_right_row_1))
print(f'左葉ノードに該当するXの行番号は：{left_u.tolist()}\n右葉ノードに該当するXの行番号は：{right_u.tolist()}')

左葉ノードに該当するXの行番号は：[0, 1]
右葉ノードに該当するXの行番号は：[2, 3, 4]


### 3-4. 左右の葉ノードの各特徴量ごとにサンプル数を振り分ける。

In [223]:
left_node_0, left_node_1, right_node_0, right_node_1 = 0, 0, 0, 0
for i in left_u:
    if y[i] == 0:
        left_node_0 += 1
    elif y[i] == 1:
        left_node_1 += 1
    else:
        pass
for j in right_u:
    if y[j] == 0:
        right_node_0 += 1
    elif y[j] == 1:
        right_node_1 += 1
    else:
        pass
print(f'左葉ノードのラベル0に該当するサンプル個数は：{left_node_0}\n左葉ノードのラベル1に該当するサンプル個数は：{left_node_1}')
print(f'右葉ノードのラベル0に該当するサンプル個数は：{right_node_0}\n右葉ノードのラベル1に該当するサンプル個数は：{right_node_1}')

左葉ノードのラベル0に該当するサンプル個数は：1
左葉ノードのラベル1に該当するサンプル個数は：1
右葉ノードのラベル0に該当するサンプル個数は：1
右葉ノードのラベル1に該当するサンプル個数は：2


### 3-5. 問題2の関数を用いて、閾値=4の情報利得を算出する。

In [224]:
gain = info_gain(left_node_0, left_node_1, right_node_0, right_node_1)
print(f'閾値：{threshold}\n情報利得：{gain:.2f}')

閾値：4
情報利得：0.01


### 3-6. 算出した情報利得を閾値として抽出した要素の元の位置と同じ場所に、同サイズの新規行列に代入する。

In [226]:
# 算出した情報利得をXと同じサイズに行列化
# 空の行列を作成する。

X_gain = np.zeros(X.shape)

# for文を用いるため、インデックスはiとjを用いるが、今回は例として閾値=4がある(1, 1)を使う。
X_gain[1, 1] = gain
X_gain

array([[0.   , 0.   ],
       [0.   , 0.013],
       [0.   , 0.   ],
       [0.   , 0.   ],
       [0.   , 0.   ]])

### 3-7. 完成した行列内からnanを除外した、最大値とその位置を算出する。

In [ ]:
max_gain = np.nanmax(X_gain)
max_index = list(zip(*np.where(X_gain == max_gain)))

### 3-8. 上記の一通りの流れを関数化する。

In [230]:
def making_array_of_gain(threshold):
    """""
    指定した閾値から、深度1に進めた後、info_gain関数を用いて情報利得を算出して、新規作成した行列に代入する。
    
    Parameters
    ----------
    threshold : int
        閾値
    return
    ----------
    gain : float
        指定した閾値の情報利得
    """""    
    
    # np.set_printoptions(precision=3)
    terminal_node_left_row_0 = list(zip(*np.where(X[:, 0] < threshold)))
    terminal_node_left_row_1 = list(zip(*np.where(X[:, 1] < threshold)))
    terminal_node_right_row_0 = list(zip(*np.where(X[:, 0] > threshold)))
    terminal_node_right_row_1 = list(zip(*np.where(X[:, 1] > threshold)))

    left_u = np.unique(np.append(terminal_node_left_row_0, terminal_node_left_row_1))
    right_u = np.unique(np.append(terminal_node_right_row_0, terminal_node_right_row_1))
    
    left_node_0, left_node_1, right_node_0, right_node_1 = 0, 0, 0, 0
    for i in range(len(left_u)):
        if y[i] == 0:
            left_node_0 += 1
        elif y[i] == 1:
            left_node_1 += 1
        else:
            pass
    for j in range(len(right_u)):
        if y[j] == 0:
            right_node_0 += 1
        elif y[j] == 1:
            right_node_1 += 1
        else:
            pass

    gain = info_gain(left_node_0, left_node_1, right_node_0, right_node_1)

    return gain

### 3-9. 全特徴量毎の全サンプル数から閾値を決めて、上記の深度1の決定木から情報利得を算出する関数を組み込んだ関数を作成して、情報利得の最大値とその位置を求める。

In [231]:
# 閾値から葉ノードへの振り分ける。
# 閾値は0列目からfor文で廻す。

X_gain = np.zeros(X.shape)
for i in range(X.shape[1]): # 各列毎
    for j in range(X.shape[0]): # 各行毎
        threshold = X[j, i]
        # print(f'threshold: {threshold}')
        gain = making_array_of_gain(threshold)
        # print(f'gain: {gain} at {j, i}')
        X_gain[j, i] = gain

max_gain = np.nanmax(X_gain)
max_index = list(zip(*np.where(X_gain == max_gain)))

### 3-10. 雛型のfit関数へ組み込む。

In [215]:
def fit(self, X, y):
    """
    全特徴量の全サンプルの中から、最適な情報利得とその位置を算出する。
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
        訓練データの特徴量
    y : 次の形のndarray, shape (n_samples, )
        訓練データの正解値
    """
    
    # 閾値は0列目からfor文で廻す。

    X_gain = np.zeros(X.shape)
    for i in range(X.shape[1]): # 各列毎
        for j in range(X.shape[0]): # 各行毎
            threshold = X[j, i]
            # print(f'threshold: {threshold}')
            gain = making_array_of_gain(threshold) # 情報利得を算出
            # print(f'gain: {gain} at {j, i}')
            X_gain[j, i] = gain

    max_gain = np.nanmax(X_gain)
    max_index = list(zip(*np.where(X_gain == max_gain)))
    if self.verbose:
        #verboseをTrueにした際は学習過程を出力
        print()
    
    return max_gain, max_index

## 【問題4】推定

推定する仕組みを実装する。

1. ScratchDecesionTreeClassifierDepth1クラスの雛形に含まれるpredictメソッドに書き加える。
2. 入力されたデータの値が、学習後の関数を用いて、どの葉ノードのどのクラスに到達するかを確認する。